In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from django.db import models
# Create your models here.

import psycopg2
#from config import config
from django.test.html import Element
import os

import os

from configparser import ConfigParser


def config(section='postgresql'):
    if os.getcwd()[0:12]!="/home/traore":
    	filename=os.getcwd()+'/database.ini'
    else:
    	filename=os.getcwd()+'/database2.ini'
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)
    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db




def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
        # create a cursor
        cur = conn.cursor()
        # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')
        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
     # close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


def read_data_sql(requete,element):
    """
    input requete outupt data
    requete='select * from chips.experiment;'
    """
    conn = None
    params = config()
    conn = psycopg2.connect(**params)
    cursor = conn.cursor()#cursor_factory=psycopg2.extras.DictCursor
    cursor.execute(requete)
    colnames = [desc[0] for desc in cursor.description]
    memory = cursor.fetchall()
    conn.close()
    cpt=0
    data=dict()
    for ele in range(len(memory)):
        cpt=cpt+1
        col=[]
        for ele1 in memory[ele]:
            col.append(str(ele1))
        dictionary ={'_key':cpt,"_value":dict(zip(colnames,col))}
        data[cpt]=dictionary
    name=element+'.csv'     
    return data

def lecture_query_tage(file_query):
    """
    function sert a lire les requetes prédefinir seulement
    """
    query_tag=dict()
    with open(file_query,"r") as fiche:
        data=fiche.readlines()
        for line in data:
            if line[0]!="#":
                if line[0:9]=="Tag_query":
                    _key=line[11:].replace('\n','').replace('"','')
                elif line[0:5]=="query":
                    query_tag[_key]=(line[6:].replace('\n','').replace('"',''))            
    return query_tag


repertoire_requete=lecture_query_tage(os.getcwd()+'/query_tag.ini')

def execution_requete(element,value_send,colname_send,order_ele=None):
    ### parametres
    if os.getcwd()[0:12]!="/home/traore":
    	my_ppty="chips"
    else:
    	my_ppty="public" #CHIPS

   
    # creation/tri des infos :(creation de structures de données)
    ###Todo
    
    # Bilan Chiffres nbr de puces par especes
    # pas Arabidopsis dans nouveau champ common_name d'array_type et 
    # analysis_type='Arrays'
    # partie requete
    my_querySpecies="select array_type_name, count(hybridization_id) "+\
    "from "+my_ppty+".hybridization "+\
    "where project_id in (select project_id from "+my_ppty+".project where is_public='yes') "+\
    "and experiment_id in (select experiment_id from "+my_ppty+".experiment "+\
    "where analysis_type='Arrays') "+\
    "and array_type_name in "+\
    "(select a.array_type_name from "+my_ppty+".array_type a where a.common_name<>'Arabidopsis')"+\
    "group by array_type_name order by array_type_name;"
    
    #Pour le nombre de ref biblio
    my_reqstat ="Select count(*) from "+my_ppty+".biblio_list;"
    
    
    #my_req_special="select o.organism_name,count(distinct ss.project_id) from "+my_ppty+".sample_source ss,"+my_ppty+".organism o where ss.project_id in( select project_id from "+my_ppty+".project where is_public='yes')  group by o.organism_name;"
    my_req_special="select o.organism_name,count(distinct ss.project_id) from "+my_ppty+".sample_source ss,"+my_ppty+".organism o where ss.project_id in( select project_id from "+my_ppty+".project ) and ss.organism_id=o.organism_id group by o.organism_name;" #where is_public='yes'
    
    my_req_recherche="SELECT * FROM global_search_element('Arabidopsis',param_schemas:=array['chips']);"
    
    my_req_recherche1="select project_id,Experiment_type,Experiment_name from "+my_ppty+".experiment where project_id in (select project_id from "+my_ppty+".project where is_public='yes');"
    
    #experiement="select * from chips.experiment where project_id in (select project_id from chips.project where is_public='yes'));"
    
    experiement="select * from (select S.project_name,O.project_id,O.experiment_name,O.experiment_type from "+my_ppty+".experiment O,"+my_ppty+".project S where O.project_id in (select project_id from "+my_ppty+".project where is_public='yes')) df limit 100;"
    
    
    try:
        my_req_recherche_av="select * from "+colname_send+" where (is_public='yes' and project_name like '%"+value_send+"%');"
    except:
        pass
    if element=='my_reqinfo':
        requete=my_reqinfo
    elif element=='my_querySpecies':
        requete=my_querySpecies
    elif element=='my_reqstat':
        requete=my_reqstat    
    elif element=='my_req_special':
        requete=my_req_special    
    elif element=='my_req_recherche':
        requete=my_req_recherche        
    elif element=='my_req_recherche_av' and colname_send!=None and value_send!=None:
        requete=my_req_recherche_av     
    elif element=='my_req_recherche1':
        requete=my_req_recherche1
    elif element=='experiement':
        requete=experiement  
    elif (element=='get_project' or element=='get_contact' or element=='my_experience' or 'my_treatment' or 'list_project' or 'type_analysis' or 'type_arrays' or 'all_experiment'):
        requete=repertoire_requete[element].replace('schema.',my_ppty+'.')
        if value_send!=None:
            requete=requete.replace('value_send',value_send)
        else:
            requete=requete
        print(requete)
    else:
        try:
            requete=eval(element)
        except:
            requete=None
            pass
    
    my_req_special="select o.organism_name,count(distinct ss.project_id) from "+my_ppty+".sample_source ss,"+my_ppty+".organism o where ss.project_id in( select project_id from "+my_ppty+".project where is_public='yes') and ss.organism_id=o.organism_id group by o.organism_name;"
    
    
    
    if  element=='my_req_special' and order_ele==None:
         requete=my_req_special  
    elif element=='my_req_special' and order_ele!=None:    
        requete="select * from ("+requete.replace(";",')')+' dfg order by dfg.'+order_ele+";"
    elif  element=='my_querySpecies' and order_ele==None:
         requete=my_querySpecies  
    elif element=='my_querySpecies' and order_ele!=None:    
        requete="select * from ("+requete.replace(";",')')+' dfg order by dfg.'+order_ele+";"
    elif  element=='my_req_recherche1' and order_ele==None:
         requete=my_req_recherche1  
    elif element=='my_req_recherche1' and order_ele!=None:    
        requete="select * from ("+requete.replace(";",')')+' dfg order by dfg.'+order_ele+";"
    if requete!=None:
        return read_data_sql(requete,element)    
    else:
        print("error") 

# def execution_requete_new(element,value_send,colname_send,order_by):
#     """
#     fonction faite pour envoyer des orders by
#     """
#     my_ppty="CHIPS"
#     
#     my_querySpecies="select * from (select array_type_name, count(hybridization_id) "+\
#     "from "+my_ppty+".hybridization "+\
#     "where project_id in (select project_id from "+my_ppty+".project where is_public='yes') "+\
#     "and experiment_id in (select experiment_id from "+my_ppty+".experiment "+\
#     "where analysis_type='Arrays') "+\
#     "and array_type_name in "+\
#     "(select a.array_type_name from "+my_ppty+".array_type a where a.common_name<>'Arabidopsis')"+\
#     "group by array_type_name order by array_type_name) pf order by pf."+order_by+";"
#     #my_req_recherche1="select project_id,Experiment_type,Experiment_name from chips.experiment where project_id in (select project_id from chips.project where is_public='yes');"
#     return read_data_sql(my_querySpecies,'text')    
#     
# 
# def execution_requete_main_A(element,value_send,colname_send):
#     my_ppty='CHIPS'
#     my_reqstat ="Select count(*) from "+my_ppty+".biblio_list;"
# #    
#    my_sample="SELECT sample.sample_name, sample.organ, sample.age_value, sample.age_unit, sample.age_db FROM "+my_ppty+".sample, "+my_ppty+".project WHERE project.project_id = sample.project_id AND project.is_public = 'yes';"
#    
#    my_sample_source="SELECT sample_source.project_id,sample_source.sample_source_name,organism.organism_name,sample_source.genotype,sample_source.mutant, sample_source.planting_date FROM "+my_ppty+".sample_source, "+my_ppty+".project , "+my_ppty+".organism, "+my_ppty+".Ecotype WHERE project.project_id=sample_source.project_id and project.is_public = 'yes' and Ecotype.ecotype_id=CAST(nullif(sample_source.ecotype_id, '') AS integer);"
#
#    requete=my_sample_source
#    if colname_send!=None:
#    	requete="select * from ("+requete.replace(';','')+") df order by "+colname_send+";" 
#    return read_data_sql(requete,element)	


In [11]:
data=execution_requete('treatment_list','','')

select treatment_type,treatment.project_id from chips.treatment,chips.project where project.project_id=treatment.project_id and project.Is_public='yes';	


In [46]:
data_comp={}
for i in data.keys():
    key=list(data[i]['_value'].keys())
    try:
            key_compt=data[i]['_value'][key[1]].split(',')
            for element in key_compt:
                if element not in data_comp:
                    data_comp[element]=[data[i]['_value'][key[0]]]
                else:
                    current=data_comp[element]
                    current.append(data[i]['_value'][key[0]])
                    data_comp[element]=current       
    except:
        pass


ImportError: No module named 'numpy'

In [43]:
data_save=[]
for el in data_comp.keys():
    data_save.append({'experiment_type':el,'count':len(data_comp[el]),'project_id':data_comp[el]})

In [60]:
data_save

[{'count': 4,
  'experiment_type': 'genetic treatment',
  'project_id': ['37', '37', '121', '147']},
 {'count': 2,
  'experiment_type': 'limited nitrogen supply',
  'project_id': ['188', '188']},
 {'count': 4,
  'experiment_type': 'syringe-inoculation',
  'project_id': ['210', '210', '210', '288']},
 {'count': 20,
  'experiment_type': 'control buffer',
  'project_id': ['25',
   '43',
   '69',
   '79',
   '97',
   '97',
   '97',
   '97',
   '97',
   '126',
   '118',
   '148',
   '165',
   '100',
   '186',
   '186',
   '193',
   '224',
   '148',
   '265']},
 {'count': 1, 'experiment_type': 'technical', 'project_id': ['244']},
 {'count': 4,
  'experiment_type': 'induction',
  'project_id': ['305', '305', '305', '297']},
 {'count': 162,
  'experiment_type': 'environmental treatment',
  'project_id': ['1',
   '0',
   '0',
   '10',
   '10',
   '15',
   '19',
   '29',
   '37',
   '37',
   '38',
   '41',
   '42',
   '42',
   '43',
   '43',
   '43',
   '43',
   '43',
   '43',
   '43',
   '42',


In [61]:
from IPython.display import display, Javascript, HTML
import json

In [62]:
%%html
<div id="d3-container"></div>
<style>
circle.node {stroke: #fff; stroke-width: 1.5px;}
line.link {stroke: #999; stroke-opacity: .6;}
</style>

In [71]:

%%javascript
require.config({
    paths: {
        d3: 'https://d3js.org/d3.v5.min'
    }
});
(function(element) {
    require(['d3'], function(d3) {   

       data = d3.csvParse('age,population
    <5,2704659
    5-13,4499890
    14-17,2159981
    18-24,3853788
    25-44,14106543
    45-64,8819342
    ≥65,612463') 
        
        var svg = d3.select(element.get(0)).append('svg')

        
 
    })
})(element);


<IPython.core.display.Javascript object>

In [53]:
#Javascript(filename='circles.js')


SyntaxError: invalid syntax (<ipython-input-53-f494f95e2ce3>, line 2)